<a href="https://colab.research.google.com/github/abdurrahmanrussel/llama2-qlora-finetune/blob/main/llama2_qlora_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Upgrade pip and install required packages
!pip install -q -U pip
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U trl
!pip install -q -U datasets safetensors


In [ ]:
from getpass import getpass
import os

# ----------------------------
# MODEL / DATA
# ----------------------------
model_name = "NousResearch/Llama-2-7b-chat-hf"  # Hugging Face gated model
dataset_name = "mlabonne/guanaco-llama2-1k"      # Instruction dataset
new_adapter_name = "llama2-7b-qlora-adapter"    # Save folder for LoRA adapter
output_dir = "/content/llama2-qlora-output"     # You can also mount Drive

# ----------------------------
# LoRA params (T4-safe)
# ----------------------------
lora_r = 32
lora_alpha = 16
lora_dropout = 0.05

# ----------------------------
# QLoRA / BitsAndBytes params
# ----------------------------
use_4bit = True
bnb_4bit_quant_type = "nf4"
bnb_4bit_compute_dtype = "float16"
use_double_quant = False

# ----------------------------
# Training params
# ----------------------------
num_train_epochs = 1
per_device_train_batch_size = 1
per_device_eval_batch_size = 1
gradient_accumulation_steps = 8  # effective batch = 1*8 = 8
learning_rate = 2e-4
weight_decay = 0.0
gradient_checkpointing = True
max_seq_length = 256  # reduce for T4 GPU
logging_steps = 25
save_steps = 200

effective_batch = per_device_train_batch_size * gradient_accumulation_steps
print(f"Effective batch size = {effective_batch}")

# Device map
device_map = "auto"

# ----------------------------
# Hugging Face token
# ----------------------------
hf_token = getpass("Enter your Hugging Face token: ")
os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer


In [ ]:
print("Loading dataset:", dataset_name)
dataset = load_dataset(dataset_name, split="train")
print(dataset)

# Automatic text field detection
text_field = None
for col in dataset.column_names:
    if col in ("text", "content", "instruction", "prompt", "messages", "input"):
        text_field = col
        break
if text_field is None:
    # fallback to first string column
    for col in dataset.column_names:
        if dataset.features[col].dtype == 'string' or str(dataset.features[col]).startswith('Value'):
            text_field = col
            break
print("Using text field:", text_field)


In [ ]:
# ----------------------------
# Step 5 - Load tokenizer + 4-bit model
# ----------------------------
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training

# Compute dtype mapping
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# BitsAndBytes 4-bit config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_double_quant,
)

# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

# Load 4-bit base model
print("Loading 4-bit model (this may take a few minutes)...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
)

model.config.use_cache = False

# Prepare for k-bit training (LoRA)
model = prepare_model_for_kbit_training(model)


In [ ]:
# ----------------------------
# Step 6 - Attach LoRA adapters (fixed)
# ----------------------------
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=["q_proj", "v_proj"],  # fix for LLaMA-2
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

# Print trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.4f}%)")


In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Training arguments (HF standard)
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    logging_steps=logging_steps,
    save_strategy="steps",
    save_steps=save_steps,
    fp16=(bnb_4bit_compute_dtype == "float16"),
    bf16=False,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    group_by_length=True,
    report_to="none",
)

# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args
)


In [ ]:
# ----------------------------
# Step 8 - Train
# ----------------------------
trainer.train()


In [ ]:
# ----------------------------
# Step 9 - Save adapter
# ----------------------------
import os

adapter_save_path = os.path.join(output_dir, new_adapter_name)
model.save_pretrained(adapter_save_path)
print("Adapter saved to:", adapter_save_path)

# Optional: merge into full model (bigger)
# merged_model = model.merge_and_unload()
# merged_model.save_pretrained(os.path.join(output_dir, "llama2-7b-merged"))


In [ ]:
# ----------------------------
# Step 10 - Load adapter + run inference
# ----------------------------
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Reload base model in 4-bit
base = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
base.config.use_cache = True

# Load LoRA adapter
model_with_adapter = PeftModel.from_pretrained(base, adapter_save_path)

# Prepare tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Prompt example
prompt = "Write a polite email asking for a meeting about QLoRA improvements."

inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model_with_adapter.device)
outputs = model_with_adapter.generate(**inputs, max_length=200)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
